In [1]:
%matplotlib widget
# %matplotlib inline
%load_ext line_profiler

import numpy as np
import seaborn as sns

from scipy import spatial, stats
from functools import reduce
from tqdm import tqdm
import matplotlib.pyplot as plt
from pprint import pprint
import itertools

In [2]:
def are_equal(kernel1, kernel2) -> bool:
    return (kernel1 == kernel2).all()

In [3]:
def variations48(cube):
    """
    List all 48 variations of the given 3d array involving both rotation and reflection
    Inspired by @Colonel Panic at https://stackoverflow.com/questions/33190042/how-to-calculate-all-24-rotations-of-3d-array
    """
    assert cube.sum() == 4

    variations = []

    def rotations4(cube, axes):
        """List the four rotations of the given 3d array in the plane spanned by the given axes."""
        for i in range(4):
            variations.append(np.rot90(cube, i, axes))

    for arr in [cube, np.flip(cube, axis=0)]:
    # for arr in [cube]:
        # imagine shape is pointing in axis 0 (up)
        # 4 rotations about axis 0
        rotations4(arr, (1,2))

        # rotate 180 about axis 1, now shape is pointing down in axis 0
        # 4 rotations about axis 0
        rotations4(np.rot90(arr, 2, axes=(0,2)), (1,2))

        # rotate 90 or 270 about axis 1, now shape is pointing in axis 2
        # 8 rotations about axis 2
        rotations4(np.rot90(arr, 1, axes=(0,2)), (0,1))
        rotations4(np.rot90(arr, -1, axes=(0,2)), (0,1))

        # rotate about axis 2, now shape is pointing in axis 1
        # 8 rotations about axis 1
        rotations4(np.rot90(arr, 1, axes=(0,1)), (0,2))
        rotations4(np.rot90(arr, -1, axes=(0,1)), (0,2))

    return variations

In [6]:
#kernels = np.empty((21, 48, 3, 3, 3), dtype=np.int8)
kernels = []

# Format: [[front],
#          [middle],
#          [back]]

# Filter 1
kernels.append(variations48(np.array([[[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 0],
                                      ],
                                      [[0, 1, 0],
                                       [1, 1, 1],
                                       [0, 0, 0],
                                      ],
                                      [[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 0],]])))

# Filter 2
kernels.append(variations48(np.array([[[0, 0, 0],
                                       [0, 0, 1],
                                       [0, 0, 0]
                                      ],
                                      [[0, 1, 0],
                                       [1, 1, 0],
                                       [0, 0, 0]
                                      ],
                                      [[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 0]]])))

# Filter 3
kernels.append(variations48(np.array([[[0, 0, 0],
                                       [0, 1, 0],
                                       [0, 0, 0]
                                      ],
                                      [[0, 1, 0],
                                       [1, 1, 0],
                                       [0, 0, 0]
                                      ],
                                      [[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 0]]])))

# Filter 4
kernels.append(variations48(np.array([[[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 0]
                                      ],
                                      [[0, 1, 0],
                                       [1, 1, 0],
                                       [0, 0, 1]
                                      ],
                                      [[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 0]]])))

# Filter 5
kernels.append(variations48(np.array([[[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 1]
                                      ],
                                      [[0, 1, 0],
                                       [1, 1, 0],
                                       [0, 0, 0]
                                      ],
                                      [[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 0]]])))

# Filter 6
kernels.append(variations48(np.array([[[0, 0, 1],
                                       [0, 0, 0],
                                       [1, 0, 0]
                                      ],
                                      [[1, 0, 0],
                                       [0, 1, 0],
                                       [0, 0, 0]
                                      ],
                                      [[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 0]]])))

# Filter 7
kernels.append(variations48(np.array([[[0, 0, 1],
                                       [0, 0, 0],
                                       [0, 0, 0]
                                      ],
                                      [[1, 0, 0],
                                       [0, 1, 0],
                                       [0, 0, 0]
                                      ],
                                      [[0, 0, 0],
                                       [0, 0, 0],
                                       [1, 0, 0]]])))

# Filter 8
kernels.append(variations48(np.array([[[0, 0, 1],
                                       [0, 0, 0],
                                       [0, 1, 0]
                                      ],
                                      [[1, 0, 0],
                                       [0, 1, 0],
                                       [0, 0, 0]
                                      ],
                                      [[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 0]]])))

# Filter 9
kernels.append(variations48(np.array([[[0, 0, 1],
                                       [0, 0, 0],
                                       [0, 0, 0]
                                      ],
                                      [[1, 0, 0],
                                       [0, 1, 0],
                                       [0, 0, 1]
                                      ],
                                      [[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 0]]])))
                                     
# Filter 10
kernels.append(variations48(np.array([[[0, 0, 1],
                                       [0, 0, 0],
                                       [0, 0, 0]
                                      ],
                                      [[1, 0, 0],
                                       [0, 1, 0],
                                       [0, 0, 0]
                                      ],
                                      [[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 1]]])))

# Filter 11
kernels.append(variations48(np.array([[[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 1]
                                      ],
                                      [[0, 0, 0],
                                       [0, 1, 0],
                                       [0, 0, 0]
                                      ],
                                      [[1, 0, 1],
                                       [0, 0, 0],
                                       [0, 0, 0]]])))

# Filter 12
kernels.append(variations48(np.array([[[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 1, 0]
                                      ],
                                      [[0, 0, 0],
                                       [0, 1, 0],
                                       [0, 0, 0]
                                      ],
                                      [[1, 0, 1],
                                       [0, 0, 0],
                                       [0, 0, 0]]])))

# Filter 13
kernels.append(variations48(np.array([[[0, 0, 1],
                                       [0, 0, 0],
                                       [0, 0, 0]
                                      ],
                                      [[0, 0, 0],
                                       [0, 1, 0],
                                       [0, 0, 0]
                                      ],
                                      [[1, 0, 1],
                                       [0, 0, 0],
                                       [0, 0, 0]]])))

# Filter 14
kernels.append(variations48(np.array([[[0, 1, 0],
                                       [0, 0, 0],
                                       [0, 0, 0]
                                      ],
                                      [[0, 0, 0],
                                       [0, 1, 0],
                                       [0, 0, 0]
                                      ],
                                      [[1, 0, 1],
                                       [0, 0, 0],
                                       [0, 0, 0]]])))

# Filter 15
kernels.append(variations48(np.array([[[0, 0, 0],
                                       [1, 0, 0],
                                       [0, 0, 0]
                                      ],
                                      [[0, 0, 0],
                                       [0, 1, 0],
                                       [0, 0, 0]
                                      ],
                                      [[1, 0, 1],
                                       [0, 0, 0],
                                       [0, 0, 0]]])))

# Filter 16
kernels.append(variations48(np.array([[[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 0]
                                      ],
                                      [[1, 0, 1],
                                       [0, 1, 0],
                                       [0, 1, 0]
                                      ],
                                      [[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 0]]])))

# Filter 17
kernels.append(variations48(np.array([[[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 1, 0]
                                      ],
                                      [[1, 0, 1],
                                       [0, 1, 0],
                                       [0, 0, 0]
                                      ],
                                      [[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 0]]])))

# Filter 18
kernels.append(variations48(np.array([[[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 1]
                                      ],
                                      [[1, 0, 1],
                                       [0, 1, 0],
                                       [0, 0, 0]
                                      ],
                                      [[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 0]]])))

# Filter 19
kernels.append(variations48(np.array([[[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 0]
                                      ],
                                      [[1, 0, 1],
                                       [0, 1, 0],
                                       [1, 0, 0]
                                      ],
                                      [[0, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 0]]])))

# Filter 20
kernels.append(variations48(np.array([[[0, 0, 1],
                                       [0, 0, 0],
                                       [0, 0, 0]
                                      ],
                                      [[0, 0, 0],
                                       [0, 1, 0],
                                       [0, 1, 0]
                                      ],
                                      [[1, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 0]]])))

# Filter 21
kernels.append(variations48(np.array([[[0, 0, 1],
                                       [0, 0, 0],
                                       [1, 0, 0]
                                      ],
                                      [[0, 0, 0],
                                       [0, 1, 0],
                                       [0, 0, 0]
                                      ],
                                      [[1, 0, 0],
                                       [0, 0, 0],
                                       [0, 0, 0]]])))

In [7]:
kernels[0]

array([[[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 1, 0],
         [1, 1, 1],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]],


       [[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 1, 0],
         [1, 1, 0],
         [0, 1, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]],


       [[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [1, 1, 1],
         [0, 1, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]],


       ...,


       [[[0, 0, 0],
         [0, 1, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [1, 1, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 1, 0],
         [0, 0, 0]]],


       [[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [1, 1, 1],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 1, 0],
         [0, 0, 0]]],


       [[[0, 0, 0],
         [0, 1, 0],
      